In [1]:
import pandas as pd

import sys
sys.path.insert(0, "..")

from src.config.paths import CHROMA_DIR, CVS_PATH_PROCESSED, JOBS_PATH_PROCESSED
from src.db_ingestion.chroma_client import get_client, get_collection, add_to_collection
from src.db_ingestion.metadata_extractor import CVMetadataExtractorCrew, JobMetadataExtractorCrew

In [2]:
# Load data
cvs_data = pd.read_csv(CVS_PATH_PROCESSED, sep=";")
jobs_data = pd.read_csv(JOBS_PATH_PROCESSED, sep=";")

In [3]:
# Init ChromaDB client
client = get_client(CHROMA_DIR)

# Get or create the "cvs" collection in ChromaDB
cvs_collection = get_collection(client, "cvs")
jobs_collection = get_collection(client, "jobs")

# Init metadata extractors
cv_crew = CVMetadataExtractorCrew()
job_crew = JobMetadataExtractorCrew()

In [4]:
# Store cvs
add_to_collection(
    metadata_extractor=CVMetadataExtractorCrew(),
    corpus=cvs_data,
    collection=cvs_collection,
    max_rpm=5,
)

2026-02-13 14:38:48.554 | INFO     | src.db_ingestion.chroma_client:add_to_collection:61 - Adding 50 documents to `cvs` collection.


 26%|██▌       | 13/50 [03:50<10:16, 16.67s/it]
2026-02-13 14:42:49.235 | WARNING  | src.db_ingestion.chroma_client:add_to_collection:83 - Null metadata keys for `doc_id=10926726`: ['country']
 78%|███████▊  | 39/50 [10:21<02:39, 14.46s/it]
2026-02-13 14:49:26.144 | WARNING  | src.db_ingestion.chroma_client:add_to_collection:83 - Null metadata keys for `doc_id=25207620`: ['country']
100%|██████████| 50/50 [13:22<00:00, 16.05s/it]


In [5]:
# Store jobs
add_to_collection(
    metadata_extractor=JobMetadataExtractorCrew(),
    corpus=jobs_data,
    collection=jobs_collection,
    max_rpm=10,
)

2026-02-13 14:54:10.116 | INFO     | src.db_ingestion.chroma_client:add_to_collection:61 - Adding 50 documents to `jobs` collection.


100%|██████████| 50/50 [08:31<00:00, 10.22s/it]


In [ ]:
# Test semantic search for cv collection
query_texts = "python machine learning engineer"

results = cvs_collection.query(
    query_texts=[query_texts],
    n_results=5,
    where={"country": "US"}
)
results

In [ ]:
# Test semantic search for job collection
query_texts = "python machine learning engineer"

results = jobs_collection.query(
    query_texts=[query_texts],
    n_results=5,
    where={"country": "US"}
)
results